In [ ]:
import os
import time
import json
from dotenv import load_dotenv
from playwright.async_api import async_playwright
from agentql import wrap, configure



# --- 1. Tải Biến Môi Trường ---
print("INFO: Đang tải biến môi trường từ .env...")
load_dotenv()

In [ ]:
# Lấy credentials từ file .env
AGENTQL_API_KEY = os.getenv("AGENTQL_API_KEY")
IDEALIST_EMAIL = os.getenv("IDEALIST_EMAIL")
IDEALIST_PASSWORD = os.getenv("IDEALIST_PASSWORD")

# Cấu hình AgentQL API Key
if AGENTQL_API_KEY:
    configure(api_key=AGENTQL_API_KEY)

# --- 2. CẤU HÌNH CHUNG ---
LOGIN_URL = "https://www.idealist.org/en/"
JOB_SEARCH_URL = "https://www.idealist.org/en/jobs"
SESSION_FILE = "idealist_login.json"
RESULT_JSON_FILE = "scraped_jobs.json"

In [ ]:
# Khởi tạo Playwright

print("--- 🚀 BẮT ĐẦU AGENTQL SCRAPER ---")
job_data_all_pages = []
headless=False         # False: Hiện browser, True: Ẩn browser
MAX_JOBS_TO_CRAWL = 5  # Số lượng jobs tối đa cần crawl
# Query dữ liệu jobs
JOB_POSTS_QUERY = """
{
    job_posts[] {
        org_name
        job_title
        salary (Optional)
        location
        contract_type (Optional)
        location_type (Optional, "remote or on-site or hybrid")
        date_posted (Optional)
    }
}
"""

PAGINATION_QUERY = """
{
    pagination {
        next_page_btn(Next page)
    }
}
"""


# --- 3. HÀM LOGIN & PASS CAPTCHA 🤖 ---
async def login_and_save_session(page):
    """
    Thực hiện quy trình login phức tạp và lưu session state.
    """
    print("INFO: 🔑 Bắt đầu quá trình login mới...")

    # 1. Đi tới URL login
    await page.goto(LOGIN_URL)

    # 2. Tìm ô email và nút "Continue"
    print("INFO: 🧠 [AgentQL] Đang tìm ô email...")
    EMAIL_QUERY = """
    {
        login_form {
            email_input
            continue_button
        }
    }
    """
    email_form = await page.query_elements(EMAIL_QUERY)

    print("INFO: 🦾 [Playwright] Đang gõ email...")
    await email_form.login_form.email_input.fill(IDEALIST_EMAIL)

    # 3. Click "Continue"
    await email_form.login_form.continue_button.click()
    print("INFO: 🦾 [Playwright] Đã click Continue. Chờ trang password...")

    # 4. Chờ và xử lý CAPTCHA nếu xuất hiện
    await asyncio.sleep(2)
    print("INFO: 🤖 Kiểm tra CAPTCHA...")
    print("INFO: ⚠️  Nếu CAPTCHA xuất hiện, vui lòng giải quyết thủ công trong 120 giây...")
    print("INFO: ⏳ Script sẽ tự động tiếp tục sau khi bạn giải CAPTCHA...")

    # 5. Chờ trang password load (timeout 120s cho CAPTCHA)
    await page.wait_for_selector('input[type="password"]', timeout=120000)
    print("INFO: ✅ CAPTCHA đã được giải quyết (hoặc không có)!")

    # 6. Tìm ô password và nút "Log in"
    print("INFO: 🧠 [AgentQL] Đang tìm ô password...")
    PASSWORD_QUERY = """
    {
        password_form {
            password_input
            login_button(Log in)
        }
    }
    """
    password_form = await page.query_elements(PASSWORD_QUERY)

    print("INFO: 🦾 [Playwright] Đang gõ password...")
    await password_form.password_form.password_input.fill(IDEALIST_PASSWORD)

    # 7. Click Login
    await password_form.password_form.login_button.click()

    # 8. Chờ login thành công
    print("INFO: ⏳ Đang chờ login thành công...")
    await asyncio.sleep(3)

    # Kiểm tra xem đã login thành công chưa
    for _ in range(20):
        current_url = page.url
        if "login" not in current_url:
            print(f"INFO: ✅ Login thành công! Đã chuyển tới: {current_url}")
            return
        await asyncio.sleep(1)

    # Nếu vẫn ở trang login sau 20s
    print(f"WARNING: ⚠️  Vẫn ở trang login. URL hiện tại: {page.url}")


# --- 4. HÀM GHI JSON RA FILE ---
def write_json_file(job_data_all_pages: list, file_path: str):
    """Ghi danh sách jobs ra file JSON."""
    try:
        with open(file_path, "w", encoding="utf-8") as f:
            json.dump(job_data_all_pages, f, ensure_ascii=False, indent=2)
        print(f"INFO: ✍️  Đã ghi {len(job_data_all_pages)} jobs ra file {file_path}")
    except Exception as e:
        print(f"ERROR: ❌ Không thể ghi file JSON: {e}")

In [ ]:
# Khởi tạo Playwright
import asyncio

async with async_playwright() as p:
    browser = await p.chromium.launch(headless=headless, slow_mo=300 if not headless else 0)
    context = None

    # --- [Kiểm tra file session] ---
    if os.path.exists(SESSION_FILE):
        try:
            print(f"INFO: 🔑 Đã tìm thấy file session ({SESSION_FILE}). Đang tải session...")
            context = await browser.new_context(storage_state=SESSION_FILE)
            print("INFO: ✅ Tải session thành công. Bỏ qua login.")
        except Exception as e:
            print(f"ERROR: ❌ Tải session thất bại: {e}. Sẽ login lại từ đầu.")
            os.remove(SESSION_FILE)
            context = None

    # --- [Login nếu chưa có session] ---
    if not context:
        context = await browser.new_context()
        page_for_login = wrap(await context.new_page())

        try:
            await login_and_save_session(page_for_login)

            # Lưu session state
            await context.storage_state(path=SESSION_FILE)
            print(f"INFO: 🔑 Đã lưu session state vào {SESSION_FILE}")

        except Exception as e:
            print(f"ERROR: ❌ Quá trình login thất bại: {e}")
            await browser.close()
            

        finally:
            await page_for_login.close()

    # --- [Scrape Jobs] ---
    page = wrap(await context.new_page())
    print(f"INFO: 🦾 [Playwright] Đang điều hướng tới: {JOB_SEARCH_URL}")
    await page.goto(JOB_SEARCH_URL)
    await asyncio.sleep(2)  # Đợi trang load

    # --- [While Loop - Scrape từng trang] ---
    page_count = 1
    while True:
        print(f"\n--- 🔄 Đang scrape trang {page_count} ---")
        print("INFO: 🧠 [AgentQL] Đang truy vấn data các job post...")

        try:
            results = await page.query_data(JOB_POSTS_QUERY, timeout=15000)
            job_list = results.get("job_posts", [])
            print("debug", job_list)
        except Exception as e:
            print(f"ERROR: ❌ Lỗi khi query data: {e}")
            break

        # Thêm jobs vào danh sách tổng
        job_data_all_pages.extend(job_list)
        print(f"INFO: ✅ Tìm thấy {len(job_list)} jobs trên trang này.")
        print(f"INFO: 📊 Tổng số jobs đã scrape: {len(job_data_all_pages)}/{MAX_JOBS_TO_CRAWL}")

        # Kiểm tra điều kiện dừng
        if len(job_list) == 0:
            print("INFO: 🤷 Không tìm thấy job nào, có thể đã hết.")
            break

        # Lưu sau mỗi trang
        write_json_file(job_data_all_pages, RESULT_JSON_FILE)

        # Tìm và click nút "Next page"
        print("INFO: 🧠 [AgentQL] Đang tìm nút 'Next page'...")
        try:
            pagination = await page.query_elements(PAGINATION_QUERY, timeout=5000)
            print("INFO: 🦾 [Playwright] Đã tìm thấy! Click để sang trang mới...")
            await pagination.pagination.next_page_btn.click()
            await page.wait_for_load_state("networkidle", timeout=20000)
            page_count += 1
            await asyncio.sleep(2)
        except:
            print("INFO: ✅ Không tìm thấy nút 'Next page'. Đã scrape hết tất cả các trang.")
            break
        
        if len(job_data_all_pages) >= MAX_JOBS_TO_CRAWL:
            print(f"INFO: 🎯 Đã đủ {MAX_JOBS_TO_CRAWL} jobs! Dừng scraping.")
            break

    # --- [KẾT THÚC] ---
    print("\n--- ✨ QUÁ TRÌNH SCRAPE HOÀN TẤT ---")
    print(f"INFO: 📈 Tổng cộng đã scrape: {len(job_data_all_pages)} jobs")

    # Ghi kết quả cuối cùng
    write_json_file(job_data_all_pages, RESULT_JSON_FILE)
    await context.close()
    await browser.close()